## Test svm classifier 

This notebook test a classifier that operates in two layers:
- First we use a SVM classifier to label utterances with high degree of certainty.
- Afterwards we use heuristics to complete the labeling

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import random
import pickle
import matplotlib.pyplot as plt

root_path = os.path.dirname(os.path.abspath(os.getcwd()))
sys.path.append(root_path)

from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from src import phase_classification as pc

data_path = os.path.join(root_path,'data')
tables_path = os.path.join(data_path,'tables')
results_path = os.path.join(root_path,'results')
output_path =os.path.join(results_path,'tables')

In [2]:
import importlib
importlib.reload(pc)

<module 'src.phase_classification' from 'C:\\Users\\CATALINA ESPINOZA\\Documents\\ciae\\Classification_IBL\\src\\phase_classification.py'>

In [3]:
WITH_STEMMING = True
#REMOVE_STOPWORDS = True
SEED = 10
NUM_TOPICS = 60
random.seed(SEED)

In [4]:
test_i = '[test1]'
file_name = test_i+'IBL_topic_distribution_by_utterance_with_phrase_before_and_after_time_utterance_minimum_0_words_with_stemming_{}_{}.xlsx'.format(WITH_STEMMING,NUM_TOPICS)
df_data = pd.read_excel(os.path.join(tables_path,'test',file_name))

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\CATALINA ESPINOZA\\Documents\\ciae\\Classification_IBL\\data\\tables\\test\\[test1]IBL_topic_distribution_by_utterance_with_phrase_before_and_after_time_utterance_minimum_0_words_with_stemming_True_60.xlsx'

In [ ]:
with open(os.path.join(data_path,'random_training.pickle'),'rb') as f:
    proportions = pickle.load(f)
    step_1 = pickle.load(f)
    step_2 = pickle.load(f)    

In [ ]:
dummy_phase = np.argmax(proportions)+1

In [ ]:
the_keys = list(set(df_data['phase']))
total_samples = 0
class_samples = {}
for key in the_keys:
    n = list(df_data.phase.values).count(key)
    #print("key {}, total {}".format(key,n))
    total_samples += n
    class_samples[key] = n
print(total_samples)
for key in the_keys:
    print("key {}, samples: {}, prop: {}".format(key,class_samples[key],round(class_samples[key]*1.0/total_samples,2)))

In [ ]:
filter_rows = list(range(0,180))+[187,188]
filter_labels = [60]

In [ ]:
all_set = pc.split_df_test(df_data)
X_all_1,y_all_1 = pc.get_data_from_dict(all_set,filter_rows)
print(len(y_all_1))
output_first_layer_1 = [dummy_phase for i in y_all_1]
labels = ["Phase {}".format(i) for i in range(1,6)]
df = pd.DataFrame(confusion_matrix(y_all_1, output_first_layer_1),columns=["Predicted {}".format(i) for i in labels])
df.index = labels
print(classification_report(y_all_1, output_first_layer_1))
df

In [ ]:
print("Accuracy {0:.3f}".format(np.sum(confusion_matrix(y_all_1, output_first_layer_1).diagonal())/len(y_all_1)))
bs = [pc.unit_vector(x) for x in y_all_1]
y_pred = [pc.unit_vector(x) for x in output_first_layer_1]
np.sqrt(np.sum([np.square(y_pred[i]-bs[i]) for i in range(len(y_all_1))])/(len(y_all_1)*2))

### Test 2

In [23]:
test_i = '[test2]'
file_name = test_i+'IBL_topic_distribution_by_utterance_with_phrase_before_and_after_time_utterance_minimum_0_words_with_stemming_{}_{}.xlsx'.format(WITH_STEMMING,NUM_TOPICS)
df_data = pd.read_excel(os.path.join(tables_path,'test',file_name))
the_keys = list(set(df_data['phase']))
total_samples = 0
class_samples = {}
for key in the_keys:
    n = list(df_data.phase.values).count(key)
    #print("key {}, total {}".format(key,n))
    total_samples += n
    class_samples[key] = n
print(total_samples)
for key in the_keys:
    print("key {}, samples: {}, prop: {}".format(key,class_samples[key],round(class_samples[key]*1.0/total_samples,2)))

100
key 1, samples: 17, prop: 0.17
key 2, samples: 6, prop: 0.06
key 3, samples: 24, prop: 0.24
key 4, samples: 1, prop: 0.01
key 5, samples: 52, prop: 0.52


In [28]:
all_set = pc.split_df_test(df_data)
X_all_2,y_all_2 = pc.get_data_from_dict(all_set,filter_rows)
t = 0.00
output_first_layer_2 = [dummy_phase for i in y_all_2]

In [31]:
labels = ["Phase {}".format(i) for i in range(1,6)]
df = pd.DataFrame(confusion_matrix(y_all_2, output_first_layer_2),columns=["Predicted {}".format(i) for i in labels])
df.index = labels
print(classification_report(y_all_2, output_first_layer_2))
df

              precision    recall  f1-score   support

           1       0.00      0.00      0.00        17
           2       0.00      0.00      0.00         6
           3       0.00      0.00      0.00        24
           4       0.00      0.00      0.00         1
           5       0.52      1.00      0.68        52

   micro avg       0.52      0.52      0.52       100
   macro avg       0.10      0.20      0.14       100
weighted avg       0.27      0.52      0.36       100



C:\Users\CATALINA ESPINOZA\AppData\Local\conda\conda\envs\teacher_topic_model\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,Predicted Phase 1,Predicted Phase 2,Predicted Phase 3,Predicted Phase 4,Predicted Phase 5
Phase 1,0,0,0,0,17
Phase 2,0,0,0,0,6
Phase 3,0,0,0,0,24
Phase 4,0,0,0,0,1
Phase 5,0,0,0,0,52


In [65]:
print("Accuracy {0:.3f}".format(np.sum(confusion_matrix(y_all_2, output_first_layer_2).diagonal())/len(y_all_2)))
bs = [pc.unit_vector(x) for x in y_all_2]
y_pred = [pc.unit_vector(x) for x in output_first_layer_2]
np.sqrt(np.sum([np.square(y_pred[i]-bs[i]) for i in range(len(y_all_2))])/(len(y_all_2)*2))

Accuracy 0.520


0.6928203230275509

In [69]:
y_all = y_all_1+y_all_2
pred = output_first_layer_1 + output_first_layer_2

In [71]:
df = pd.DataFrame(confusion_matrix(y_all, pred),columns=["Predicted {}".format(i) for i in labels])
df.index = labels
print(classification_report(y_all, pred))
df

              precision    recall  f1-score   support

           1       0.00      0.00      0.00        72
           2       0.00      0.00      0.00        33
           3       0.00      0.00      0.00        70
           4       0.00      0.00      0.00         8
           5       0.35      1.00      0.52        98

   micro avg       0.35      0.35      0.35       281
   macro avg       0.07      0.20      0.10       281
weighted avg       0.12      0.35      0.18       281



C:\Users\CATALINA ESPINOZA\AppData\Local\conda\conda\envs\teacher_topic_model\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,Predicted Phase 1,Predicted Phase 2,Predicted Phase 3,Predicted Phase 4,Predicted Phase 5
Phase 1,0,0,0,0,72
Phase 2,0,0,0,0,33
Phase 3,0,0,0,0,70
Phase 4,0,0,0,0,8
Phase 5,0,0,0,0,98


In [70]:
print("Accuracy {0:.3f}".format(np.sum(confusion_matrix(y_all, pred).diagonal())/len(y_all)))
bs = [pc.unit_vector(x) for x in y_all]
y_pred = [pc.unit_vector(x) for x in pred]
np.sqrt(np.sum([np.square(y_pred[i]-bs[i]) for i in range(len(y_all))])/(len(y_all)*2))

Accuracy 0.349


0.8069978634429111